In [3]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('../Benchmark/GoogleNews-vectors-negative300.bin', binary=True)


In [75]:
from pyemd import emd  
import gensim, math
import time
import pandas as pd
import nltk 
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import lda
import numpy as np
import gensim
from nltk import word_tokenize
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
import string
from scipy.spatial.distance import cosine
from gensim.matutils import softcossim
from scipy.sparse import csc_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import mixture
from collections import Counter

# Data sets

In [55]:
from sklearn.datasets import fetch_rcv1
rcv1 = fetch_rcv1()


KeyError: (1,)

In [7]:
np.mean([[1,2,3],[2,3,4]], axis = 0)

array([ 1.5,  2.5,  3.5])

In [32]:


texts = ["economics money finance policy politics economy",
         "cat cow dog tiger lion elephant monkey",
         "microeconomics consumption market stock currency production",
        "Obama speaks to the media in Illinois",
         "The president greets the press in Chicago",
        "It's sunny outside, let's have lunch in the sun.",
        "the weather is really nice, why don't we eat outside",
        "snail tastes not aweful, french people are weird",
        "It's sunny outside.",
        "Let's have lunch in the sun.",
         "Artificial intelligence (AI) makes it possible for machines to learn from experience, \
         adjust to new inputs and perform human-like tasks. Most AI examples that you hear about today – from \
         chess-playing computers to self-driving cars – rely heavily on deep learning and natural language \
         processing. Using these technologies, computers can be trained to accomplish specific tasks by processing \
         large amounts of data and recognizing patterns in the data.The term artificial intelligence was coined in \
         1956, but AI has become more popular today thanks to increased data volumes, advanced algorithms, and \
         improvements in computing power and storage.AI adds intelligence to existing products. In most cases, AI \
         will not be sold as an individual application. Rather, products you already use will be improved with AI \
         capabilities, much like Siri was added as a feature to a new generation of Apple products. Automation, \
         conversational platforms, bots and smart machines can be combined with large amounts of data to improve \
         many technologies at home and in the workplace, from security intelligence to investment analysis.",
         "Artificial intelligence (AI, also machine intelligence, MI) is intelligence demonstrated by machines, \
in contrast to the natural intelligence (NI) displayed by humans and other animals. In computer science AI research \
is defined as the study of intelligent agents: any device that perceives its environment and takes actions that \
maximize its chance of successfully achieving its goals. Colloquially, the term artificial intelligence is applied \
when a machine mimics cognitive functions that humans associate with other human minds, such as learning and \
problem solving The scope of AI is disputed: as machines become increasingly capable, tasks considered as requiring \
intelligence are often removed from the definition, a phenomenon known as the AI effect, leading to the quip, \
AI is whatever hasn't been done yet. For instance, optical character recognition is frequently excluded from \
artificial intelligence, having become a routine technology. Capabilities generally classified as AI as of 2017 \
include successfully understanding human speech, competing at the highest level in strategic game systems (such as \
chess and Go), autonomous cars, intelligent routing in content delivery network and military simulations. \
Artificial intelligence was founded as an academic discipline in 1956, and in the years since has experienced several \
waves of optimism followed by disappointment and the loss of funding (known as an AI winter), followed by new \
approaches, success and renewed funding. For most of its history, AI research has been divided into subfields that \
often fail to communicate with each other. These sub-fields are based on technical considerations, such as particular \
goals (e.g. robotics or machine learning), the use of particular tools (logic or neural networks), or deep \
philosophical differences. Subfields have also been based on social factors (particular institutions or the work of \
particular researchers). The traditional problems (or goals) of AI research include reasoning, knowledge \
representation, planning, learning, natural language processing, perception and the ability to move and manipulate \
objects. General intelligence is among the field's long-term goals. Approaches include statistical methods, \
computational intelligence, and traditional symbolic AI. Many tools are used in AI, including versions of search and \
mathematical optimization, neural networks and methods based on statistics, probability and economics. The AI field \
draws upon computer science, mathematics, psychology, linguistics, philosophy and many others.The field was founded \
on the claim that human intelligence can be so precisely described that a machine can be made to simulate it This \
raises philosophical arguments about the nature of the mind and the ethics of creating artificial beings endowed with \
human-like intelligence which are issues that have been explored by myth, fiction and philosophy since antiquity. \
Some people also consider AI to be a danger to humanity if it progresses unabatedly\
Others believe that AI, unlike previous technological revolutions, will create a risk of mass unemployment.\
In the twenty-first century, AI techniques have experienced a resurgence following concurrent advances in computer \
power, large amounts of data, and theoretical understanding; and AI techniques have become an essential part of the \
technology industry, helping to solve many challenging problems in computer science."]

In [ ]:
cluster_

In [120]:
#gmm = mixture.BayesianGaussianMixture(n_components=num_clusters, covariance_type='full', 
#                              init_params = 'kmeans', 
#                              weight_concentration_prior_type = 'dirichlet_process').fit(vecs)
#labels = gmm.predict(vecs)

#km = KMeans(n_clusters = num_clusters)
#km.fit(vecs)
start = time.time()
text = texts[-1]
x = texts[-1]
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space    
text = text.translate(translator).lower().split()
text = [t for t in text if t in word2vec ]
wordSum = len(text)
word_counter = Counter(text)
unique_text = list(word_counter.keys())
word_counts = list(word_counter.values())
num_clusters = int(np.sqrt(len(unique_text)))
#print(num_clusters)
vecs = [word2vec[w] for w in unique_text]
print(time.time() - start)
start = time.time()
sc = SpectralClustering(num_clusters, affinity='precomputed', 
                        assign_labels = "discretize", n_init=100)
similarity_matrix = cosine_similarity(vecs)
sc.fit(similarity_matrix)
print(time.time() - start)
start = time.time()
silhoutte_scores = silhouette_samples(1-similarity_matrix, sc.labels_, metric = 'precomputed')
indices = [np.where(sc.labels_ == i)[0].tolist() for i in range(num_clusters)]

sh_means = [np.mean(np.take(silhoutte_scores, indices[i])) for i in range(num_clusters)]
sh_weights = np.array(sh_means/sum(sh_means))

sizes_weights = np.array([sum(np.take(word_counts, indices[i]))/wordSum for i in range(num_clusters)])
p = 0.3
final_weights = p*sh_weights+(1-p)*sizes_weights
#print("{},{},{}".format(sum(sh_weights), sum(sizes_weights), sum(final_weights)))
centroids = np.array([np.mean(np.take(vecs, indices[i], axis = 0), axis = 0) for i in range(num_clusters)])
rep = np.concatenate([[num_clusters], centroids.flatten(), final_weights])
print(time.time() - start)
#print("{},{}".format(len(text), len(vecs)))

for i in range(num_clusters):
    print(final_weights[i])
    print("{}".format(np.take(unique_text, np.where(sc.labels_ == i)[0].tolist())))

0.001291036605834961
0.020720243453979492
1.0
0.009849786758422852
0.0753917171674
['computer' 'agents' 'device' 'functions' 'tasks' 'requiring' 'optical'
 'technology' 'capabilities' 'systems' 'routing' 'content' 'delivery'
 'network' 'use' 'tools' 'networks' 'processing' 'methods' 'versions'
 'search' 'optimization' 'techniques' 'data']
0.0279815642478
['successfully' 'founded' 'years' 'has' 'success' 'history' 'long' 'twenty'
 'century' 'resurgence' 'industry']
0.0417589472524
['demonstrated' 'displayed' 'applied' 'associate' 'considered' 'removed'
 'known' 'recognition' 'excluded' 'autonomous' 'divided' 'based'
 'representation' 'manipulate' 'used' 'described' 'endowed']
0.0935779264586
['is' 'become' 'hasn' 'been' 'done' 'yet' 'was' 'or' 'have' 'work' 'can'
 'be' 'so' 'it' 'this' 'about' 'if' 'believe']
0.119445345998
['also' 'other' 'as' 'such' 'increasingly' 'are' 'often' 'frequently'
 'generally' 'include' 'several' 'most' 'these' 'traditional' 'among'
 'many' 'including' 'othe

In [95]:
for i in range(100):
    if math.ceil(np.sqrt(i)) >= i:
        print(i)

0
1
2


# Topic soft cosine distance
build_vec() transforms a text into a vector.
Cluster_TSCD() computes the soft cosine similarity between two text vectors.

In [204]:
#similarity_matrix = np.ones((len(allTopics), len(allTopics)), np.float64)
#for i in range(1, len(allTopics)):
#    for j in range(i):
#        x = time.time()
#        distance_matrix[i, j] = euclidean(allTopics[i], allTopics[j])
#        similarity_matrix[i, j] = 1-cosine(allTopics[i], allTopics[j])
#        similarity_matrix[j, i] = similarity_matrix[i, j]
def build_vec_auto(text, cls):
    x = text
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space    
    text = text.translate(translator).lower().split()
    start = time.time()
    text = [t for t in text if t in word2vec and not t in stopwords.words("english")]
    if cls == "kmeans":
        num_clusters = math.ceil(np.sqrt(len(set(text))))
        vecs = [word2vec[w] for w in text]
        km = KMeans(n_clusters = num_clusters, random_state=3425, precompute_distances = True)
        km.fit(vecs)
        cluster_sizes = np.array([len(np.where(km.labels_ == i)[0]) for i in range(num_clusters)])
        weights = cluster_sizes/sum(cluster_sizes)
        return np.concatenate([[num_clusters], km.cluster_centers_.flatten(), weights])
    elif cls == "spectral":
        wordSum = len(text)
        word_count = Counter(text)
        unique_text = list(word_count.keys())
        if len(unique_text) < 3:
            unique_text = unique_text*2
        num_clusters = math.ceil(np.sqrt(len(unique_text)))
        #print(num_clusters)
        vecs = [word2vec[w] for w in unique_text]
        sc = SpectralClustering(num_clusters, affinity='precomputed', random_state=3425,
                                assign_labels = "discretize", n_init=100)
        similarity_matrix = cosine_similarity(vecs)+1
        sc.fit(similarity_matrix)
        silhoutte_scores = silhouette_samples(1-similarity_matrix, sc.labels_, metric = 'precomputed')
        indices = [np.where(sc.labels_ == i)[0].tolist() for i in range(num_clusters)]
        sh_means = [np.mean(np.take(silhoutte_scores, indices[i])) for i in range(num_clusters)]
        sh_weights = np.array(sh_means/sum(sh_means))
        sizes_weights = np.array([sum([word_count[word] 
                 for word in np.take(unique_text, indices[i])])/wordSum for i in range(num_clusters)])
        p = 0.5
        weights = p*sh_weights+(1-p)*sizes_weights
        #print("{},{},{}".format(sum(sh_weights), sum(sizes_weights), sum(final_weights)))
        centroids = np.array([np.mean(np.take(vecs, indices[i], axis = 0), axis = 0) for i in range(num_clusters)])
        rep = np.concatenate([[num_clusters], centroids.flatten(), weights])
        print(rep)
        return rep
        
    

def cluster_TSCD_auto(a, b):
    def euclidean(a, b):
        return np.sqrt(np.sum(np.square(a-b)))
    start = time.time()
    num_clusters_a, num_clusters_b = int(a[0]), int(b[0])
    cut_a, cut_b = 1+300*num_clusters_a, 1+300*num_clusters_b
    topics1, topics2 = a[1:cut_a].reshape(-1,300), b[1:cut_b].reshape(-1,300)
    weights1, weights2 = a[cut_a:cut_a+num_clusters_a], b[cut_b:cut_b+num_clusters_b]
    allTopics = np.concatenate([topics1,topics2])
    d1 = np.concatenate([weights1,[0]*len(topics2)])
    d2 = np.concatenate([[0]*len(topics1),weights2])
    similarity_matrix = cosine_similarity(allTopics)
    result = d1.T.dot(similarity_matrix).dot(d2)
    vec1 = d1.T.dot(similarity_matrix).dot(d1)
    vec2 = d2.T.dot(similarity_matrix).dot(d2)
    result /= (np.sqrt(vec1)*np.sqrt(vec2))
    #print(time.time()-start)
    return np.clip(1-result, 0 , 1)

def build_vec(text, num_clusters):
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
    text = text.translate(translator).lower().split()
    vecs = [word2vec[w] for w in text if w in word2vec and not w in stopwords.words("english")]
    km = KMeans(n_clusters = num_clusters, random_state=3425, precompute_distances = True)
    km.fit(vecs)
    cluster_sizes = [len(np.where(km.labels_ == i)[0]) for i in range(num_clusters)]
    return np.append(km.cluster_centers_.flatten(), np.array(cluster_sizes))

def cluster_TSCD(a, b, num_clusters):
    def euclidean(a, b):
        return np.sqrt(np.sum(np.square(a-b)))
    cut = 300*num_clusters
    topics1, topics2 = a[0:cut].reshape(-1,300), b[0:cut].reshape(-1,300)
    freq1, freq2 = a[cut:], b[cut:]
    allTopics = np.concatenate([topics1,topics2])
    
    d1 = np.concatenate([freq1/sum(freq1),[0]*len(topics2)])
    d2 = np.concatenate([[0]*len(topics1),freq2/sum(freq2)])
    similarity_matrix = np.ones((len(allTopics), len(allTopics)), np.float64)
    for i in range(1, len(allTopics)):
        for j in range(i):
            #distance_matrix[i, j] = euclidean(allTopics[i], allTopics[j])
            similarity_matrix[i, j] = 1-cosine(allTopics[i], allTopics[j])
            similarity_matrix[j, i] = similarity_matrix[i, j]
    result = d1.T.dot(similarity_matrix).dot(d2)
    vec1 = d1.T.dot(similarity_matrix).dot(d1)
    vec2 = d2.T.dot(similarity_matrix).dot(d2)
    result /= (np.sqrt(vec1)*np.sqrt(vec2))
    return np.clip(1-result, 0, 1)

def cluster_TSCD_raw(text1, text2, num_clusters):
    return cluster_TSCD(build_vec(text1, num_clusters), build_vec(text2, num_clusters), num_clusters)

def cluster_TSCD_raw_auto(text1, text2, cls):
    return cluster_TSCD_auto(build_vec_auto(text1, cls), build_vec_auto(text2, cls))

start = time.time()

num_clusters = 2
#vecs = [build_vec_auto(t, "kmeans") for t in texts]
#print(time.time() - start)
start = time.time()
vecs = [build_vec_auto(t, "spectral") for t in texts]
#print(time.time() - start)
#for i in range(1000):
    #cluster_TSCD_auto(vecs[10], vecs[11])
    #print(cluster_WMDdistance(texts[0], texts[i], num_clusters))
#start = time.time()
#for i in range(100):
    #build_vec_auto(texts[-1], "spectral")
#print(time.time() - start)
start = time.time()
#for i in range(100):
    #build_vec_auto(texts[-1], "kmeans")
#print(time.time() - start)
#print(time.time() - start)
#cluster_TSCD_auto(vecs[10], vecs[11])
start = time.time()
print(cluster_TSCD_raw_auto(texts[-1], texts[-2], "spectral"))
print("time:{}".format(time.time() - start))
start = time.time()
print(cluster_TSCD_raw_auto(texts[-1], texts[-2], "kmeans"))
print("time:{}".format(time.time() - start))
print(cluster_WMDdistance(texts[3], texts[4], 3))

/Users/schen/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:205: RuntimeWarning: divide by zero encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)


[  3.00000000e+00   1.73339844e-01  -9.52148438e-03  -4.73632812e-02
   1.24755859e-01  -7.20214844e-03  -1.17568970e-01  -8.08410645e-02
  -1.25976562e-01   1.79687500e-01  -8.17260742e-02  -9.76562500e-04
   6.62231445e-02  -3.38134766e-02   1.01089478e-01  -1.99707031e-01
   2.80761719e-01  -2.19726562e-03   2.42675781e-01   4.67529297e-02
   5.68847656e-02   6.59179688e-03   2.16308594e-01   2.07519531e-01
   8.61816406e-02   2.61535645e-02  -5.00488281e-02  -1.53808594e-02
   3.49121094e-02   1.17187500e-01  -9.00878906e-02  -1.38671875e-01
  -2.16674805e-01  -3.39355469e-02  -8.20312500e-02   1.24023438e-01
  -1.14257812e-01   2.45117188e-01   3.96728516e-03  -7.28759766e-02
   2.25463867e-01   7.57446289e-02  -3.78417969e-02   2.84179688e-01
  -3.41796875e-02  -1.09375000e-01  -8.19549561e-02   6.29882812e-02
   2.52441406e-01  -2.55859375e-01   1.56250000e-01   1.71142578e-01
   1.52832031e-01   1.31835938e-02  -2.35839844e-01  -1.46484375e-02
  -2.24304199e-02  -5.20019531e-02

[ 16.          -0.06688097   0.10746935 ...,   0.04803861   0.07016724
   0.04401825]
[ 16.          -0.06688097   0.10746935 ...,   0.04803861   0.07016724
   0.04401825]
[ 10.           0.04057312   0.04160156 ...,   0.10366327   0.09624154
   0.11473512]
0.122954765861
time:0.20504403114318848
0.0851600298281
time:0.31360793113708496
2.0989535225578897


# Clustered WMD distance. 
Takes raw text inputs.

In [122]:

def cluster_WMDdistance(text1, text2, num_clusters):
    
    def build_rep(text, num_clusters):
        translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
        text = text.translate(translator).lower().split()
        vecs = [word2vec[w] for w in text if w in word2vec]
        km = KMeans(n_clusters = num_clusters, random_state=3425)
        km.fit(vecs)
        cluster_sizes = [len(np.where(km.labels_ == i)[0]) for i in range(num_clusters)]
        return km.cluster_centers_, np.array(cluster_sizes)
        
    def euclidean(a, b):
        return np.sqrt(np.sum(np.square(a-b)))

    topics1, freq1 = build_rep(text1, num_clusters)
    topics2, freq2 = build_rep(text2, num_clusters)
    allTopics = np.concatenate([topics1,topics2])
    d1 = np.concatenate([freq1/sum(freq1),[0]*len(topics2)])
    d2 = np.concatenate([[0]*len(topics1),freq2/sum(freq2)]) 
    distance_matrix = [ [euclidean(allTopics[i], allTopics[j]) 
                        for j in range(len(allTopics))] 
                           for i in range(len(allTopics)) ]
    #print(distance_matrix)
    return emd(d1, d2, np.array(distance_matrix))




# Word average distance


In [ ]:
def wad(text):
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
    text = text.translate(translator).lower().split()
    text = [t for t in text if not t in stopwords.words("english")]
    return np.mean([word2vec[w] for w in text if w in word2vec], axis=0)


In [194]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
cats = ['comp.graphics',
        'sci.crypt',
        'rec.autos',
        'sci.space']
        #'talk.politics.guns',
        #'rec.sport.baseball']
news20_train = fetch_20newsgroups(subset='train',categories=cats)
news20_test = fetch_20newsgroups(subset='test',categories=cats)

print("started vectorization")
start = time.time()
train_vecs = [ build_vec_auto(t,"spectral") for t in news20_train.data]
test_vecs = [build_vec_auto(t, "spectral") for t in news20_test.data]
max_len = np.max(np.concatenate(([len(a) for a in train_vecs], [len(a) for a in test_vecs])))
train_vecs = np.asarray([np.pad(a, (0, max_len - len(a)), 'constant', constant_values=0) for a in train_vecs])
test_vecs = np.asarray([np.pad(a, (0, max_len - len(a)), 'constant', constant_values=0) for a in test_vecs])
#train_vecs = [wad(t) for t in news20_train.data]
print(time.time() - start)

started vectorization


/Users/schen/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/schen/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/Users/schen/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/Users/schen/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:205: RuntimeWarning: divide by zero encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)


256.67481994628906


In [202]:
train_vecs

array([[  9.00000000e+00,   4.88281250e-04,   1.22734919e-01, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  1.00000000e+01,   4.27024141e-02,  -3.57402451e-02, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  1.00000000e+01,  -8.98666382e-02,   1.49078369e-02, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       ..., 
       [  2.20000000e+01,   6.11223504e-02,  -2.10461214e-01, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  7.00000000e+00,  -1.27964020e-01,   6.06079102e-02, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  1.20000000e+01,  -1.06066249e-01,   4.75656614e-02, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [201]:
print("started KNN ")
start = time.time()
#nbrs = GridSearchCV(KNeighborsClassifier(algorithm='ball_tree'), cv = None, 
     #               param_grid={"n_neighbors": range(5,25,5)})
#nbrs = GridSearchCV(KNeighborsClassifier(algorithm='ball_tree', 
 #                           metric=lambda a,b: cluster_TSCD_auto(a, b)), cv = None,
  #                         param_grid={"n_neighbors": range(5,25,5)})
nbrs = KNeighborsClassifier(algorithm='ball_tree', n_neighbors = 15,
                            metric=lambda a,b: cluster_TSCD_auto(a, b))
nbrs.fit(train_vecs, news20_train.target)
print(time.time() - start)
start = time.time()
#test_vecs = [build_vec(t, num_clusters) for t in news20_test.data]
#test_vecs = [ wad(t) for t in news20_test.data]
print("started predicting")
start = time.time()
predicted = nbrs.predict(test_vecs)
print(accuracy_score(predicted, news20_test.target))
print(time.time() - start)

started KNN 


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [26]:
start = time.time()
df = pd.DataFrame(train_vecs)
df = df.fillna(0)
print(time.time() - start)
start = time.time()
max_len = np.max([len(a) for a in train_vecs])
x = np.asarray([np.pad(a, (0, max_len - len(a)), 'constant', constant_values=0) for a in train_vecs])
print(time.time() - start)
#x
len(test_vecs)

0.6671648025512695
0.29393792152404785


1187

In [ ]:
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
text = news20_train.data[0].translate(translator).lower().split()
print(news20_train.data[0])
text

In [ ]:
for i in range(0,1,1):
    print(i)
    

In [ ]:
x = np.array([0,1,1])
x.dot(np.array([[1,0,1],[0,1,1],[0,1,0]]))